# AzureML Studio : Automated ML

In this notebook, we will use the AzureML Studio's [AutoML](https://docs.microsoft.com/en-us/azure/machine-learning/concept-automated-ml) to automatically select the best model given time and compute constraints.

The AutoML process is as follows:
- check the data for potential issues (Class balancing detection, Missing feature values imputation, High cardinality feature detection)
- build data pipelines with :
  - a data preprocessor (scaling, standardisation, normalisation)
  - a prediction model (XGBoost, LightGBM, Random Forest, Linear Regression, etc.)
  - a set of hyperparameters (learning rate, number of trees, etc.)
- train each pipeline for a limited time
- evaluate the performance of each pipeline
- select the best pipeline

The experiment is visible in the AzureML Studio : [oc-p7-automated-ml](https://ml.azure.com/experiments/id/9bde22d7-75e6-41cc-9bfe-03e15873f292?wsid=/subscriptions/da2e4791-6dd1-422b-848a-a961cef6ab89/resourcegroups/OC_P7/workspaces/oc-p7-ml-workspace&tid=43204f6d-c600-4585-985a-6bafda08d2bb)

We will compare this pre-trained local model to the baseline model from [1_baseline.ipynb](1_baseline.ipynb).

## AutoML model : max 1h training on CPU

In this version, we did not include DNN models in the AutoML process, because they require GPU resources.

This AutoML run is available in the AzureML Studio : [automl_1h-cpu](https://ml.azure.com/experiments/id/9bde22d7-75e6-41cc-9bfe-03e15873f292/runs/AutoML_d6a52352-b3dd-4cf3-82e3-b5e44d038436?wsid=/subscriptions/da2e4791-6dd1-422b-848a-a961cef6ab89/resourcegroups/OC_P7/workspaces/oc-p7-ml-workspace&tid=43204f6d-c600-4585-985a-6bafda08d2bb#models)

Here are the models that were trained in the AutoML process :

![AzureML - AutomatedML - 1h on CPU - models](img/azureml_automated_ml_1h_cpu_models.png)

### Best model

The best model is a `XGBoostClassifier` (wrapper for [XGBClassifier](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier)) with [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html) .

![Best Model](img/azureml_automated_ml_1h_cpu_best_model.png)


### Results

| Confusion Matrix | Precision Recall Curve (AP = 0.79) | ROC Curve (AUC = 0.80) |
|---|---|---|
| ![Confusion Matrix](img/azureml_automated_ml_1h_cpu_confusion_matrix.png) | ![Precision Recall Curve](img/azureml_automated_ml_1h_cpu_precision_recall.png) | ![ROC Curve](img/azureml_automated_ml_1h_cpu_ROC.png) |


The performances on the dataset are quite better than our baseline model : 
- Average Precision = 0.79 (baseline = 0.73 , +8.2%)
- ROC AUC = 0.80 (baseline = 0.74 , +8.1%)

Unlike our baseline model, this model is quite balanced, just slightly biased towards the _POSITIVE_ class. It is much less biased than our baseline model : it predicted 9% (baseline = 35% , -74%) more _POSITIVE_ (78403) messages than _NEGATIVE_ (65597).


## AutoML model : max 10h training on GPU


This AutoML run is available in the AzureML Studio : [automl_10h-gpu](https://ml.azure.com/experiments/id/9bde22d7-75e6-41cc-9bfe-03e15873f292/runs/AutoML_cfd7047e-a8bb-459a-b31d-976a48d42381?wsid=/subscriptions/da2e4791-6dd1-422b-848a-a961cef6ab89/resourcegroups/OC_P7/workspaces/oc-p7-ml-workspace&tid=43204f6d-c600-4585-985a-6bafda08d2bb#models)

Here are the models that were trained in the AutoML process :

![AzureML - AutomatedML - 10h on GPU - models](img/azureml_automated_ml_10h_gpu_models.png)

### Best model

The best model is a [LightGBM](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html) with [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html).

This model adds a pre-processing step, which [integrates and fine-tunes a pre-trained BERT model](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-features#bert-integration-in-automated-ml), before training the actual classification model : [`PretrainedTextDNNTransformer`](https://docs.microsoft.com/en-us/python/api/azureml-automl-runtime/azureml.automl.runtime.featurizer.transformer.text.pretrained_text_dnn_transformer.pretrainedtextdnntransformer?view=azure-ml-py).

![Best Model](img/azureml_automated_ml_10h_gpu_best_model.png)


### Results

| Confusion Matrix | Precision Recall Curve (AP = 0.942) | ROC Curve (AUC = 0.942) |
|---|---|---|
| ![Confusion Matrix](img/azureml_automated_ml_10h_gpu_confusion_matrix.png) | ![Precision Recall Curve](img/azureml_automated_ml_10h_gpu_precision_recall.png) | ![ROC Curve](img/azureml_automated_ml_10h_gpu_ROC.png) |


The performances on the dataset are quite better than our previous model : 
- Average Precision = 0.94 (baseline = 0.73 , +29%)
- ROC AUC = 0.94 (baseline = 0.74 , +27%)

Like our previous model, this model is very fair, just very slightly biased towards the _NEGATIVE_ class this time. It is much less biased than our baseline model : it predicted only 1.4% (baseline = 35% , -96%) more _NEGATIVE_ (128909) messages than _POSITIVE_ (127091).
